In [1]:
!pip install torch torchvision torchaudio numpy transformers tokenizers torchtext matplotlib pypdf langchain

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'D:\\Cortech\\Learning\\youtube-stuffs\\env\\Lib\\site-packages\\matplotlib-3.10.0.dist-info\\RECORD2o9zp8ye.tmp' -> 'D:\\Cortech\\Learning\\youtube-stuffs\\env\\Lib\\site-packages\\matplotlib-3.10.0.dist-info\\RECORD'
Check the permissions.




  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/1.9 MB ? eta -:--:--
   ---------- ----------------------------- 0.5/1.9 MB 799.2 kB/s eta 0:00:02
   ---------- ----------------------------- 0.5/1.9 MB 799.2 kB/s eta 0:00:02
   ---------------- ----------------------- 0.8/1.9 MB 838.9 kB/s eta 0:00:02
   --------------------- ------------------ 1.0/1.9 MB 838.4 kB/s eta 0:00:02
   -------------------------- ------------- 1.3/1.9 MB 838.9 kB/s eta 0:00:01
   -------------------------- ------------- 1.3/1.9 MB 838.9 kB/s eta 0:00:01
   -------------------------------- ------- 1.6/1.9 MB 838.6 kB/s eta 0:00:01
   ------------------------------------- -- 1.8/1.9 MB 846.1 kB/s eta 0:00:01
   ---------------------------------------- 1.9/1.9 MB 850.4 kB/s eta 0:00:00
   -----------------------

In [1]:
from langchain.document_loaders import PyPDFLoader

# Load the PDF document
loader = PyPDFLoader('data/gpt4all.pdf')
doc = loader.load_and_split()

# Check the number of documents and preview the first one
print("Number of documents:", len(doc))
doc[0]

Number of documents: 9


Document(metadata={'source': 'data/gpt4all.pdf', 'page': 0}, page_content='GPT4All: An Ecosystem of Open Source Compressed Language Models\nYuvanesh Anand\nNomic AI\nyuvanesh@nomic.ai\nZach Nussbaum\nNomic AI\nzach@nomic.ai\nAdam Treat\nNomic AI\nadam@nomic.ai\nAaron Miller\nNomic AI\naaron@nomic.ai\nRichard Guo\nNomic AI\nrichard@nomic.ai\nBen Schmidt\nNomic AI\nben@nomic.ai\nGPT4All Community\nPlanet Earth\nBrandon Duderstadt∗\nNomic AI\nbrandon@nomic.ai\nAndriy Mulyar∗\nNomic AI\nandriy@nomic.ai\nAbstract\nLarge language models (LLMs) have recently\nachieved human-level performance on a range\nof professional and academic benchmarks. The\naccessibility of these models has lagged behind\ntheir performance. State-of-the-art LLMs re-\nquire costly infrastructure; are only accessible\nvia rate-limited, geo-locked, and censored web\ninterfaces; and lack publicly available code and\ntechnical reports.\nIn this paper, we tell the story of GPT4All, a\npopular open source repository that aim

In [5]:
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace

# Initialize tokenizer
vocab_size = 5000
tokenizer = Tokenizer(BPE())
tokenizer.pre_tokenizer = Whitespace()
# Train tokenizer on the text from the document
trainer = BpeTrainer(vocab_size = vocab_size-5)
special_token_dict = {
    "<start>" : 0,
    "<|end|>": 1,
    "<|system|>": 2, 
    "<|user|>": 3,
    "<|ai|>": 4
}
tokenizer.add_special_tokens(list(special_token_dict))
# trainer = BpeTrainer(vocab_size=vocab_size, special_tokens=["<start>", "<|end|>", "<system>", "<|user|>", "<|ai|>"])
text_data = [doc_chunk.page_content for doc_chunk in doc]  # Collect text from all chunks
print(text_data)
text_list = [text_data]
tokenizer.train_from_iterator(text_list, trainer)

# Save and reload tokenizer (optional)
print("tokenizer trained")
print("length of tokenizer", len(tokenizer.get_vocab()))
# tokenizer.save("tokenizer.json")
# tokenizer = Tokenizer.from_file("tokenizer.json")

# Define encode and decode functions
def encode(text):
    return tokenizer.encode(text).ids

def decode(ids):
    return tokenizer.decode(ids)

# Test the tokenizer
sample_text = text_data[0][:200]  # Use the first 200 characters of the first document chunk
encoded = encode(sample_text)
decoded = decode(encoded)

print("Encoded sample:", encoded)
print("Decoded sample:", decoded)

['GPT4All: An Ecosystem of Open Source Compressed Language Models\nYuvanesh Anand\nNomic AI\nyuvanesh@nomic.ai\nZach Nussbaum\nNomic AI\nzach@nomic.ai\nAdam Treat\nNomic AI\nadam@nomic.ai\nAaron Miller\nNomic AI\naaron@nomic.ai\nRichard Guo\nNomic AI\nrichard@nomic.ai\nBen Schmidt\nNomic AI\nben@nomic.ai\nGPT4All Community\nPlanet Earth\nBrandon Duderstadt∗\nNomic AI\nbrandon@nomic.ai\nAndriy Mulyar∗\nNomic AI\nandriy@nomic.ai\nAbstract\nLarge language models (LLMs) have recently\nachieved human-level performance on a range\nof professional and academic benchmarks. The\naccessibility of these models has lagged behind\ntheir performance. State-of-the-art LLMs re-\nquire costly infrastructure; are only accessible\nvia rate-limited, geo-locked, and censored web\ninterfaces; and lack publicly available code and\ntechnical reports.\nIn this paper, we tell the story of GPT4All, a\npopular open source repository that aims to\ndemocratize access to LLMs. We outline the\ntechnical details of th

In [6]:
import torch
from torch.utils.data import Dataset

# Custom dataset class
class TextDataset(Dataset):
    def __init__(self, data, context_length):
        self.data = torch.tensor(data, dtype=torch.long)
        self.context_length = context_length

    def __len__(self):
        return len(self.data) - self.context_length

    def __getitem__(self, idx):
        x = self.data[idx:idx + self.context_length]
        y = self.data[idx + 1:idx + self.context_length + 1]
        return x, y

# Tokenize the full text
all_text = " ".join(text_data)
print(all_text)
encoded_data = encode(all_text)

# Split into train/validation sets
context_length = 32
train_size = int(0.9 * len(encoded_data))
train_data = encoded_data[:train_size]
val_data = encoded_data[train_size:]

# Create datasets
train_dataset = TextDataset(train_data, context_length)
val_dataset = TextDataset(val_data, context_length)

print("Training dataset size:", len(train_dataset))
print("Validation dataset size:", len(val_dataset))

GPT4All: An Ecosystem of Open Source Compressed Language Models
Yuvanesh Anand
Nomic AI
yuvanesh@nomic.ai
Zach Nussbaum
Nomic AI
zach@nomic.ai
Adam Treat
Nomic AI
adam@nomic.ai
Aaron Miller
Nomic AI
aaron@nomic.ai
Richard Guo
Nomic AI
richard@nomic.ai
Ben Schmidt
Nomic AI
ben@nomic.ai
GPT4All Community
Planet Earth
Brandon Duderstadt∗
Nomic AI
brandon@nomic.ai
Andriy Mulyar∗
Nomic AI
andriy@nomic.ai
Abstract
Large language models (LLMs) have recently
achieved human-level performance on a range
of professional and academic benchmarks. The
accessibility of these models has lagged behind
their performance. State-of-the-art LLMs re-
quire costly infrastructure; are only accessible
via rate-limited, geo-locked, and censored web
interfaces; and lack publicly available code and
technical reports.
In this paper, we tell the story of GPT4All, a
popular open source repository that aims to
democratize access to LLMs. We outline the
technical details of the original GPT4All model
family, as well a

In [7]:
from torch.utils.data import DataLoader

# Create DataLoaders
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Inspect a sample batch
sample_batch = next(iter(train_loader))
print("Input batch shape:", sample_batch[0].shape)
print("Target batch shape:", sample_batch[1].shape)

Input batch shape: torch.Size([16, 32])
Target batch shape: torch.Size([16, 32])


In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

# Define the Transformer model components
class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(embedding_dim, head_size)
        self.query = nn.Linear(embedding_dim, head_size)
        self.value = nn.Linear(embedding_dim, head_size)
        self.dropout = nn.Dropout(0.1)

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)
        v = self.value(x)
        # Compute attention scores
        wei = q @ k.transpose(-2, -1) / np.sqrt(k.size(-1))
        mask = torch.triu(torch.ones(T, T), diagonal=1).bool().to(device)
        wei = wei.masked_fill(mask, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        # Apply attention to values
        out = wei @ v
        return out

class MultiHeadAttention(nn.Module):
    def __init__(self, head_size, num_heads):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, embedding_dim)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.proj(out)
        return out

class FeedForward(nn.Module):
    def __init__(self, hidden_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim * 4),
            nn.ReLU(),
            nn.Linear(hidden_dim * 4, hidden_dim)
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    def __init__(self, hidden_dim, num_heads):
        super().__init__()
        head_size = hidden_dim // num_heads
        self.attention = MultiHeadAttention(head_size, num_heads)
        self.ff = FeedForward(hidden_dim)
        self.ln1 = nn.LayerNorm(hidden_dim)
        self.ln2 = nn.LayerNorm(hidden_dim)

    def forward(self, x):
        x = x + self.attention(self.ln1(x))
        x = x + self.ff(self.ln2(x))
        return x

# Define the Small Language Model
embedding_dim = 128
num_heads = 12
num_layers = 4
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



class SmallLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding = nn.Embedding(vocab_size, embedding_dim)
        self.pos_embedding = nn.Embedding(context_length, embedding_dim)
        self.blocks = nn.Sequential(*[Block(embedding_dim, num_heads) for _ in range(num_layers)])
        self.ln = nn.LayerNorm(embedding_dim)
        self.lm_head = nn.Linear(embedding_dim, vocab_size)

    def forward(self, x, targets=None):
        B, T = x.shape
        tok_emb = self.token_embedding(x)
        pos_emb = self.pos_embedding(torch.arange(T, device=device))
        x = tok_emb + pos_emb

        # Apply Transformer block 
        x = self.blocks(x)
        x = self.ln(x)
        logits = self.lm_head(x)
        
        if targets is None:
            return logits, None
        else:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)
            return logits, loss

    @torch.no_grad()
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -context_length:]
            logits, _ = self(idx_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat([idx, idx_next], dim=1)
        return idx


embedding_dim = 128
num_heads = 12
num_layers = 4
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SmallLanguageModel().to(device)

print(model)


In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# Training loop
num_steps = 2000
for step in range(num_steps):
    for x, y in train_loader:
        x, y = get_batch('train')
        x, y = x.to(device), y.to(device)
        logits, loss = model(x, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if step % 100 == 0:
        print(f"Step {step}, Loss: {loss.item():.4f}")

    if step % 1000 == 0:
        torch.save(model.state_dict(), f"model_step_{step}.pth")

Step 0, Loss: 1.4065


In [ ]:
# Text generation
start_prompt = "Once upon a time"
encoded_prompt = torch.tensor([encode(start_prompt)], dtype=torch.long).to(device)
generated = model.generate(encoded_prompt, max_new_tokens=50)
print("Generated text:", decode(generated[0].tolist()))